In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from operator import itemgetter

In [2]:
model = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()
parser = StrOutputParser()

In [4]:
loader = PyMuPDFLoader("ref_data/TWINGO/MR413X4416A000.pdf", extract_images=True)
pages = loader.load()
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [3]:
template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [8]:
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [9]:
question = """
Что делать если  Стартер не функционирует
"""

In [10]:
for s in chain.stream({"question": question}):
    print(s, end="", flush=True)

Если стартер не функционирует, выполните следующие действия:

1. Проверьте состояние аккумуляторной батареи (см. главу 80A, Аккумуляторная батарея).
2. Проверьте состояние силового предохранителя (см. ПРОВЕРКА 2).
3. Проверьте исправность предохранителя F9 компонента 1016.
4. Проверьте, не заклинило ли двигатель.
5. Проверьте надежность затяжки следующих наконечников:
   - аккумуляторную батарею,
   - стартера,
   - "массы" двигателя,
   - "массы" кузова.
6. Затяните клеммы (см. Руководство по ремонту 411, глава 80A, Аккумуляторная батарея).
7. Проверьте исправность следующих проводов:
   - провод, соединяющий вывод "+" аккумуляторной батареи с контактом B "+" стартера,
   - провод, соединяющий вывод "-" аккумуляторной батареи с "массой" кузова автомобиля,
   - провод, соединяющий двигатель с "массой" кузова автомобиля.
8. Устраните неисправность поврежденных проводов и их наконечников. Если цепь неисправна и есть возможность ремонта, устраните неисправность. Если ремонта нет, замените